In [1]:

import livecellx
from livecellx.model_zoo.segmentation.custom_transforms import CustomTransformEdtV9
from livecellx.preprocess.utils import overlay


import numpy as np
import json
from livecellx.core import (
    SingleCellTrajectory,
    SingleCellStatic,
    SingleCellTrajectoryCollection,
)
from livecellx.core.single_cell import get_time2scs
from livecellx.core.datasets import LiveCellImageDataset
from livecellx.preprocess.utils import (
    overlay,
    enhance_contrast,
    normalize_img_to_uint8,
)
import matplotlib.pyplot as plt
import os
from pathlib import Path
import pandas as pd
from typing import List


import glob
from PIL import Image, ImageSequence
import numpy as np
from matplotlib import pyplot as plt
from pathlib import Path
import json


import torch
import torch
import torch.utils.data
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import TQDMProgressBar

from livecellx.model_zoo.segmentation.sc_correction import CorrectSegNet
from livecellx.model_zoo.segmentation.sc_correction_aux import CorrectSegNetAux
from livecellx.core.utils import label_mask_to_edt_mask
from livecellx.core.single_cell import combine_scs_label_masks
import livecellx.model_zoo.segmentation.csn_configs as csn_configs
import numpy as np
import warnings
import tqdm
from livecellx.core.io_sc import prep_scs_from_mask_dataset
from livecellx.core.sc_filters import filter_boundary_cells, filter_scs_by_size
import random

import tqdm
from datetime import datetime
import torch
from livecellx.core.utils import label_mask_to_edt_mask
from livecellx.segment.ou_viz import viz_ou_outputs
from livecellx.segment.ou_utils import create_ou_input_from_sc
from livecellx.preprocess.utils import normalize_edt
from livecellx.core.io_utils import save_png
from livecellx.model_zoo.segmentation.sc_correction_dataset import CorrectSegNetDataset
from livecellx.model_zoo.segmentation.csn_sc_utils import correct_sc, correct_sc_mask


from livecellx.segment.ou_utils import dilate_or_erode_label_mask



from livecellx.model_zoo.segmentation.sc_correction_dataset import CorrectSegNetDataset

torch.manual_seed(237)
# PADDING_PIXELS = 50
# PADDING_PIXELS = 10
PADDING_PIXELS = 20
OUT_THRESHOLD = 1


model_ckpt = "/home/ken67/livecellx/notebooks/lightning_logs/version_v18_02-inEDTv1-augEdtV9-scaleV2-lr-0.0001-aux-seed-404/checkpoints/last.ckpt"

# model_ckpt = (
#     "lightning_logs/version_v17_02-inEDTv1-augEdtV8-scaleV2-lr=0.00001-aux/checkpoints/last.ckpt"
# )
model = CorrectSegNetAux.load_from_checkpoint(model_ckpt)
# input_transforms = csn_configs.gen_train_transform_edt_v8(degrees=0, shear=0, flip_p=0)

model.cuda()
model.eval()



input_transforms = CustomTransformEdtV9(degrees=0, shear=0, flip_p=0, use_gaussian_blur=True, gaussian_blur_sigma=15)


lcx_out_dir = Path("./notebook_results/CXA_process2_7_19/")
lcx_out_dir.mkdir(parents=True, exist_ok=True)
# mapping_path = lcx_out_dir / "iomin_all_sci2sci2metric.json"
mapping_path = lcx_out_dir / "iou_all_sci2sci2metric.json"


sci2sci2metric = json.load(open(mapping_path, "r"))



# all_scs = prep_scs_from_mask_dataset(d2_mask_dataset, d2_mask_dataset)
all_tracked_scs = SingleCellTrajectoryCollection.load_from_json_file(
    lcx_out_dir / "sctc_filled_SORT_bbox_max_age_3_min_hits_1.json").get_all_scs()

filtered_tracked_scs = filter_boundary_cells(all_tracked_scs, dist_to_boundary=30, use_box_center=False)


print("# of cp_scs", len(all_tracked_scs))
print("# of filtered_cp_scs", len(filtered_tracked_scs))
print("# of filtered cells", len(all_tracked_scs) - len(filtered_tracked_scs))

# ## Apply CSN to all the cells



2025-01-19 20:09:27.219145: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-19 20:09:27.219175: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-19 20:09:27.220036: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-19 20:09:27.224270: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-19 20:09:27.847836: W tensorflow/compiler/tf2

>>> Using MSE loss
>>> Based on loss type, training output threshold:  1
|-----> json loaded from notebook_results/CXA_process2_7_19/sctc_filled_SORT_bbox_max_age_3_min_hits_1.json
|-----> Creating SingleCellTrajectoryCollection from json_dict...
|-----> Loading SingleCellTrajectoryCollection from json_dict done.
# of cp_scs 134553
# of filtered_cp_scs 106348
# of filtered cells 28205


In [2]:
id2sc = {sc.id: sc for sc in all_tracked_scs}

# Use only the date in the filename
csn_out = lcx_out_dir / "csn_apply_all" / f"v18-404-correct-ALL-scs-{datetime.now().strftime('%Y%m%d')}_PAD={PADDING_PIXELS}_OUTTH={OUT_THRESHOLD}"


csn_viz_out = csn_out / "viz"
csn_mask_out = csn_out / "mask"
csn_out.mkdir(parents=True, exist_ok=True)
csn_viz_out.mkdir(parents=True, exist_ok=True)
csn_mask_out.mkdir(parents=True, exist_ok=True)

results_df = pd.DataFrame(
    columns=["sc_id", "label_str", "#ws-scs"]
)
results_dict = {
    "sc_id": [],
    "label_str": [],
    "#ws-scs": [],
}
save_df_interval = 100

all_corrected_scs = []
for idx, _sc in tqdm.tqdm(enumerate(filtered_tracked_scs), total=len(filtered_tracked_scs)):
    # out_mask_transformed, watershed_mask, label_str = correct_sc_mask(sc_from, model, PADDING_PIXELS, input_transforms, gpu=True)
    res_dict = correct_sc(_sc, model, PADDING_PIXELS, input_transforms, gpu=True, return_outputs=True)
    _corrected_scs = res_dict["scs"]
    all_corrected_scs.extend(_corrected_scs)

    out_mask_transformed = res_dict["out_mask"]
    watershed_mask = res_dict["watershed_mask"]
    label_str = res_dict["label_str"]
    
    _sc.meta["csn_out_aux_label"] = label_str

    results_dict["sc_id"].append(_sc.id)
    results_dict["label_str"].append(label_str)
    results_dict["#ws-scs"].append(len(np.unique(watershed_mask)) - 1)
    # # # Save resaults
    save_png(out_mask_transformed[0], csn_mask_out / f"raw-seg-{_sc.id}_from.png", mode="RGB") # 3 channel, (SEG, US, OS)
    save_png(watershed_mask, csn_mask_out / f"watershed-sc-{_sc.id}_from.png")

    np.save(csn_mask_out / f"raw-seg-{_sc.id}.npy", out_mask_transformed[0], allow_pickle=False)
    np.save(csn_mask_out / f"watershed_sc-{_sc.id}.npy", watershed_mask, allow_pickle=False)
    if idx % save_df_interval == 0:
        results_df = pd.DataFrame(results_dict)
        results_df.to_csv(csn_out / "results.csv", index=False)

SingleCellStatic.write_single_cells_json(all_corrected_scs, csn_out / "csn_applied_scs.json")




  0%|          | 0/106348 [00:00<?, ?it/s]/home/ken67/anaconda3/envs/livecellx/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/home/ken67/anaconda3/envs/livecellx/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResi

In [ ]:
# SORT track
from livecellx.track.sort_tracker_utils import track_SORT_bbox_from_scs

img_dataset = all_corrected_scs[0].img_dataset
mask_dataset = all_corrected_scs[0].mask_dataset
all_corrected_scs = filter_scs_by_size(all_corrected_scs, min_size=400)

tracked_sctc = track_SORT_bbox_from_scs(all_corrected_scs, img_dataset, mask_dataset, max_age=3, min_hits=1)


In [13]:
tracked_sctc.write_json(csn_out / "corrected_all_retrack_v18-1-19.json")